In [0]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
wine_data = pd.read_csv('/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset/wine.csv')

In [4]:
wine_data.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,0,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,0,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,0,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,0,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [5]:
wine_data.shape

(178, 14)

In [6]:
wine_data.sample(8)

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
20,0,14.06,1.63,2.28,16.0,126,3.00,3.17,0.24,2.10,5.65,1.09,3.71,780
177,2,14.13,4.10,2.74,24.5,96,2.05,0.76,0.56,1.35,9.20,0.61,1.60,560
52,0,13.82,1.75,2.42,14.0,111,3.88,3.74,0.32,1.87,7.05,1.01,3.26,1190
75,1,11.66,1.88,1.92,16.0,97,1.61,1.57,0.34,1.15,3.80,1.23,2.14,428
50,0,13.05,1.73,2.04,12.4,92,2.72,3.27,0.17,2.91,7.20,1.12,2.91,1150
29,0,14.02,1.68,2.21,16.0,96,2.65,2.33,0.26,1.98,4.70,1.04,3.59,1035
0,0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
47,0,13.90,1.68,2.12,16.0,101,3.10,3.39,0.21,2.14,6.10,0.91,3.33,985


In [7]:
wine_features = wine_data.drop('Class', axis=1)
wine_features.sample(8)

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
46,14.38,3.59,2.28,16.0,102,3.25,3.17,0.27,2.19,4.90,1.04,3.44,1065
74,11.96,1.09,2.30,21.0,101,3.38,2.14,0.13,1.65,3.21,0.99,3.13,886
136,12.25,4.72,2.54,21.0,89,1.38,0.47,0.53,0.80,3.85,0.75,1.27,720
94,11.62,1.99,2.28,18.0,98,3.02,2.26,0.17,1.35,3.25,1.16,2.96,345
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
58,13.72,1.43,2.50,16.7,108,3.40,3.67,0.19,2.04,6.80,0.89,2.87,1285
65,12.37,1.21,2.56,18.1,98,2.42,2.65,0.37,2.08,4.60,1.19,2.30,678
107,12.72,1.75,2.28,22.5,84,1.38,1.76,0.48,1.63,3.30,0.88,2.42,488


In [8]:
wine_target = wine_data[['Class']]
wine_target.sample(5)

,Class
15,0
120,1
149,2
107,1
72,1


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(wine_features, wine_target, test_size=0.4, random_state=0)

In [10]:
X_train.shape, Y_train.shape

((106, 13), (106, 1))

In [0]:
X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(X_test.values).float()

In [12]:
X_train.shape, X_test.shape

(torch.Size([106, 13]), torch.Size([72, 13]))

In [0]:
Y_train = torch.from_numpy(Y_train.values).view(1,-1)[0]
Y_test = torch.from_numpy(Y_test.values).view(1,-1)[0]

In [14]:
Y_train.shape, Y_test.shape

(torch.Size([106]), torch.Size([72]))

In [0]:
input_size = 13
output_size = 3
hidden_size = 100

In [0]:
class Net(nn.Module):

  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fc3 = nn.Linear(hidden_size, output_size)

  def forward(self, X):
    X = torch.sigmoid(self.fc1(X))
    X = torch.sigmoid(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [0]:
model = Net()

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr = 0.01)

loss_fn = nn.NLLLoss()

In [0]:
epochs = 1000

In [20]:
for epoch in range(epochs):

  optimizer.zero_grad()
  Y_pred = model(X_train)

  loss = loss_fn(Y_pred, Y_train)
  loss.backward()

  optimizer.step()

  if epoch%100 == 0:
    print('Epoch : ', epoch, ' Loss : ', loss.item())

Epoch :  0  Loss :  1.1083730459213257
Epoch :  100  Loss :  0.2725227475166321
Epoch :  200  Loss :  0.1992054581642151
Epoch :  300  Loss :  0.0427107997238636
Epoch :  400  Loss :  0.03635503351688385
Epoch :  500  Loss :  0.03384397551417351
Epoch :  600  Loss :  0.03217708319425583
Epoch :  700  Loss :  0.03089394047856331
Epoch :  800  Loss :  0.029771754518151283
Epoch :  900  Loss :  0.028471240773797035


In [21]:
model.state_dict

<bound method Module.state_dict of Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)>

In [22]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.0820,  0.0365,  0.1825,  ..., -0.1929, -0.0132,  0.2207],
                      [-0.1826, -0.1811,  0.2170,  ..., -0.1283,  0.1835,  0.1049],
                      [ 0.2356,  0.1473, -0.1996,  ..., -0.1075, -0.1872, -0.2618],
                      ...,
                      [ 0.1229, -0.2130,  0.1868,  ..., -0.0745, -0.0675, -0.2241],
                      [-0.0425,  0.0500,  0.1303,  ..., -0.0845, -0.1100,  0.2615],
                      [ 0.1823, -0.2399,  0.0433,  ...,  0.2523, -0.2106,  0.1548]])),
             ('fc1.bias',
              tensor([-0.0901, -0.0834, -0.0110, -0.2185,  0.1710, -0.0484, -0.2192,  0.1567,
                       0.0515, -0.0661, -0.0487, -0.0583,  0.1573, -0.0810,  0.0103,  0.0367,
                       0.2283,  0.1092, -0.1011,  0.1463, -0.2664, -0.1358,  0.0099, -0.2386,
                       0.2043,  0.0947, -0.2019,  0.0272,  0.0391, -0.0369, -0.1018,  0.0589,
                       0.1874,  0.04

In [23]:
optimizer.state_dict

<bound method Optimizer.state_dict of Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)>

In [24]:
optimizer.state_dict()

{'param_groups': [{'amsgrad': False,
   'betas': (0.9, 0.999),
   'eps': 1e-08,
   'lr': 0.01,
   'params': [139804118556464,
    139804118556176,
    139804311322840,
    139804311322768,
    139804311325504,
    139804311324136],
   'weight_decay': 0}],
 'state': {139804118556176: {'exp_avg': tensor([ 0.0000e+00, -5.6052e-45,  5.6052e-45, -1.3200e-18, -5.6052e-45,
            2.4416e-26,  0.0000e+00, -4.2599e-43, -7.9152e-41, -6.6642e-35,
            2.5960e-22, -5.6052e-45,  0.0000e+00,  1.0359e-18,  0.0000e+00,
           -5.6052e-45, -6.4482e-15,  0.0000e+00, -5.9419e-28, -6.7323e-19,
           -2.1108e-21, -1.1701e-27,  0.0000e+00,  9.8022e-21, -5.6052e-45,
           -8.6040e-43,  1.3135e-25,  0.0000e+00, -4.8111e-34,  0.0000e+00,
            6.6499e-21, -5.6052e-45,  0.0000e+00,  0.0000e+00,  5.7662e-36,
            0.0000e+00, -2.8950e-18,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  4.6173e-04,
            0.0000e+00,

#More robust way to save parameters of a model

In [25]:
cd "/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset/"

/content/drive/My Drive/Deep_Learning_Projects/Wine_Dataset


In [0]:
torch.save(model.state_dict(), "models/wine_classifier_state_dict")

In [0]:
new_model = Net()

In [30]:
new_model.load_state_dict(torch.load('models/wine_classifier_state_dict'))

<All keys matched successfully>

In [31]:
new_model.eval()

Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=3, bias=True)
)

In [0]:
predict_out = new_model(X_test)
_, predict_y = torch.max(predict_out, 1)

In [0]:
from sklearn.metrics import precision_score, accuracy_score, recall_score

In [34]:
print("Accuracy score : ", accuracy_score(Y_test.data, predict_y.data))
print("Precision score : ", precision_score(Y_test.data, predict_y.data, average='micro'))
print("Recall score : ", recall_score(Y_test.data, predict_y.data, average='micro'))

Accuracy score :  0.9305555555555556
Precision score :  0.9305555555555556
Recall score :  0.9305555555555556
